# Pruebas RIS
### <font color='5E5D5D'> Descripción </font>

<i><font color='C9614B'> En esta notebook se encuentran las
pruebas realizadas para validar el funcionamiento de la métrica y su utilización (Experimentos 5 y 6.1). Tener en cuenta que la configuración que se presenta aquí, representa únicamente una de las evaluadas.</font></i><br>

***

## Configuración

#### Librerías

In [4]:
# Librerías del sistema que son prerequsito: descomentar estas líneas únicamente si se necesita su instalación
! pip install gym pyvirtualdisplay > /dev/null 2>&1
! apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

# Modelos
from models.models import DQNModel, NFQModel

# Agentes
from agents.dqn_agent import DQNAgent
from agents.nfq_agent import NFQAgent

# Utilitarios
from utils.utils_func import process_state, save_dataset, load_dataset
from utils.visualization import plot_results, wrap_env, show_video
from utils.metrics import ris_behavior_policy, ris_estimator

# Gym
import gym

# PyTorch
import torch

# Misc
import numpy as np
import random

# Plots
import matplotlib.pyplot as plt

# Sofmax
from scipy.special import softmax

## Pruebas para Experimentos 5 y 6.1

La configuración planteada a continuación es la realizada para el experimento 5. Para realizar el expermiento 6 se realizó la configuración mencionada en la documentación.

### Deep Q-Learning

Entrenamiento de agentes DQN, utilizando el ambiente ``MountainCar-v0``.

#### Mountain Car

In [5]:
# Variables globales de entrenamiento
BUFFER_SIZE = 2000
GAMMA = 0.99
NUM_EPISODES = 1000
MAX_STEPS = 200

BATCH_SIZE = 64
LEARNING_RATE = 0.001

# Generación de datasets (RAND, ESA, EMA)
NUM_SAMPLES = 1000
NUM_RUNS = 1
DATASET_ACTION_TYPE='greedy'

In [ ]:
# Arrays para los resultados finales
rewards_mc_dqn, steps_mc_dqn = [], []
agent_mc_dqn = None  

# Modelo para agente DQN
model_dqn = None

# Dataset para guardar las experiencias
dataset = []

# Lista para almacenar las trayectorias (para el cálculo de RIS)
trajectories = []

# Seed inicial
num_seed = 42

for _run in range(NUM_RUNS):
  print(f"\nRun #{_run+1} | Seed: {num_seed}")
  print("********************************************************")

  # Creo el ambiente
  env = gym.make("MountainCar-v0")

  # Seed
  env.seed(num_seed)
  random.seed(num_seed)
  np.random.seed(num_seed)
  torch.manual_seed(num_seed)
  torch.backends.cudnn.deterministic = True
  
  # Creo el modelo
  model_dqn = DQNModel(2, env.action_space.n)

  # Creo el agente
  agent_mc_dqn = DQNAgent(env, model_dqn, process_state, 
                          BUFFER_SIZE, BATCH_SIZE, LEARNING_RATE, GAMMA, 
                          epsilon_i=0.99, epsilon_f=0.1, epsilon_anneal_time=1000)

  # Entreno al agente
  rewards, steps_per_episode = agent_mc_dqn.train(NUM_EPISODES, MAX_STEPS)
  
  # Resultados
  rewards_mc_dqn.append(rewards)
  steps_mc_dqn.append(steps_per_episode)

  # Genero dataset a partir de la red entrenada
  env = gym.make("MountainCar-v0")
  experience, traject = agent_mc_dqn.generate_dataset(env, action_type=DATASET_ACTION_TYPE, epsilon=.1, num_samples=NUM_SAMPLES, max_steps=MAX_STEPS)
  dataset.extend(experience)
  trajectories.extend(traject)

  # Incremento el seed
  num_seed += 1

# Promedio de resultados por corrida
rewards_mc_dqn = np.mean(rewards_mc_dqn, axis=0)
steps_mc_dqn = np.mean(steps_mc_dqn, axis=0)

In [9]:
# Guardar el dataset de experiencia en formato pickle
filename = 'datasets/dataset_sample.pkl'
save_dataset(filename, dataset)

In [10]:
# Guardar el dataset de trayectorias en formato pickle
filename = 'datasets/trajectories_sample.pkl'
save_dataset(filename, trajectories)

### Neural Fitted Q-Iteration

Entrenamiento de agentes NFQ, utilizando el ambiente ``MountainCar-v0``.

#### Mountain Car

In [11]:
# Cargar el dataset de experiencia desde archivo pickle
filename = 'datasets/dataset_sample.pkl'
dataset = load_dataset(filename)

In [12]:
# Cargar el dataset de trayectorias desde archivo pickle
filename = 'datasets/trajectories_sample.pkl'
trajectories = load_dataset(filename)

In [13]:
# Variables globales de entrenamiento
GAMMA = 0.99
NUM_EPISODES = 1000
MAX_STEPS = 200
BATCH_SIZE = 64
LEARNING_RATE = 0.001

# Variables para RIS
N_ACTIONS = 3
HORIZON = 0

In [14]:
# Arrays para los resultados finales
rewards_mc_nfq, steps_mc_nfq = [], []

# Creo el ambiente
env = gym.make("MountainCar-v0")

# Creo el modelo
model_nfq = NFQModel(2, env.action_space.n)

# Creo el agente
agent_mc_nfq = NFQAgent(env, model_nfq, process_state, 
                        BATCH_SIZE, LEARNING_RATE, GAMMA,
                        dataset, trajectories, 
                        N_ACTIONS, HORIZON)

In [ ]:
# Entreno al agente
rewards, steps_per_episode, ris_per_episode = agent_mc_nfq.train_from_dataset(NUM_EPISODES)

In [ ]:
# Gráfico de RIS
plt.plot(range(NUM_EPISODES), ris_per_episode)
plt.title("Episode RIS")
plt.xlabel("Episode Number")
plt.ylabel("Value")
plt.show()

## Validación del indicador


Pruebas para validar el funcionamiento del indicador RIS en diferentes casos según lo mencionado en el experimento 5

In [17]:
# Cantidad de trayectorias
paths_number = len(agent_mc_nfq.trajectories.memory)

# Definición de behavior_policy
behavior_policy = ris_behavior_policy(agent_mc_nfq.trajectories.memory, agent_mc_nfq.n_actions, paths_number, agent_mc_nfq.horizon)


In [ ]:
# Resultado RIS
ris = ris_estimator(agent_mc_nfq.trajectories.memory,
                    agent_mc_dqn.device,
                    agent_mc_nfq.policy_net,
                    behavior_policy,
                    agent_mc_nfq.n_actions,
                    agent_mc_nfq.horizon,
                    n=None,
                    weighted=False)
ris

In [19]:
# Definiciones
trajectories_memory = agent_mc_nfq.trajectories.memory
PATHS_NUMBERS = len(trajectories_memory)
n_actions = 3
HORIZON = 1
device_ris = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
action_type = DATASET_ACTION_TYPE 

In [20]:
# Definiciones de las políticas a utilizar
pi_DQN = model_dqn
pi_NFQ = model_nfq

estimated_pi_DQN = ris_behavior_policy(trajectories_memory,
                               N_ACTIONS,
                               PATHS_NUMBERS,
                               HORIZON)                               

In [21]:
# Función implementada para verificar la implementación del indicador
def pruebas_ris(trajectories_memory, device_ris, evaluation_policy, paths_numbers, behavior_policy, action_type, horizon):
  
  # Inicializar los pesos y retorno
  ws = np.ones(paths_numbers)
  gs = np.zeros(paths_numbers)

  # Loop
  for x, path in enumerate(trajectories_memory):
    long_ob = []

    states, actions, rewards, _, _ = list(zip(*path))
    gs[x] = sum(rewards)

    for state, action in zip(states, actions):
      state = tuple(state.numpy())
      long_ob.append(state)
      state = (torch.tensor(state)).to(device_ris)

      # Calcular valor estado-accion para evaluation policy (p) según las distintas opciones
      if isinstance(evaluation_policy, dict):
        p = evaluation_policy[tuple(long_ob)][action]
      elif evaluation_policy == pi_NFQ:
        p = evaluation_policy(state).view(-1,1).squeeze().detach()
        p = softmax(p)
        p = p[action].numpy()
      else:
        # Evaluar si la transición es un dummy (transiciones agregadas para completar max_steps)
        if torch.all(state.eq(torch.tensor([0.55, 0]))).item():
          p = 1
        else:
          p = evaluation_policy(state).view(-1,1).squeeze().detach()
          p = softmax(p)
          maximum_p = max(p)
          p = torch.tensor(list(map(lambda x: 1 if x == maximum_p else 0, p)))
          p = p[action].numpy()
    
      # Calcular valor estado-accion para behavior policy (q) según las distintas opciones
      if isinstance(behavior_policy, dict):
        q = behavior_policy[tuple(long_ob)][action]
      elif behavior_policy == pi_NFQ:
        q = behavior_policy(state).view(-1,1).squeeze().detach()
        q = softmax(q)
        q = q[action].numpy()

      else:
        # Evaluar si la transición es un dummy (transiciones agregadas para completar max_steps)
        if torch.all(state.eq(torch.tensor([0.55, 0]))).item():
          q = 1
        else:
          q = behavior_policy(state).view(-1,1).squeeze().detach()
          q = softmax(q)
          maximum_q = max(q)
          q = torch.tensor(list(map(lambda x: 1 if x == maximum_q else 0, q)))
          q = q[action].numpy()
        

      # Calcular pesos    
      ws[x] *= p/q

      # Agregar la acción
      long_ob.append(action)

      # Chequear Horizonte
      if len(long_ob) >= horizon * 2:
        long_ob = long_ob[2:]

  # Calcular valor RIS(n)
  ris = np.dot(gs, ws)
  ris = ris / paths_numbers

  return ris, ws, gs

In [ ]:
# Prueba: policy evaluation = pi_NFQ, policy behavior = pi_NFQ
pruebas_ris(trajectories_memory,
            device_ris,
            pi_NFQ,
            PATHS_NUMBERS,
            pi_NFQ,
            action_type,
            HORIZON)

In [ ]:
# Prueba: policy evaluation = pi_DQN, policy behavior = pi_DQN
pruebas_ris(trajectories_memory,
            device_ris,
            pi_DQN,
            PATHS_NUMBERS,
            pi_DQN,
            action_type,
            HORIZON)

In [ ]:
# Prueba: policy evaluation = estimated_pi_DQN, policy behavior = estimated_pi_DQN
pruebas_ris(trajectories_memory,
            device_ris,
            estimated_pi_DQN,
            PATHS_NUMBERS,
            estimated_pi_DQN,
            action_type,
            HORIZON)

In [ ]:
# Prueba: policy evaluation = pi_NFQ, policy behavior = pi_DQN
pruebas_ris(trajectories_memory,
            device_ris,
            pi_NFQ,
            PATHS_NUMBERS,
            pi_DQN,
            action_type,
            HORIZON)

In [ ]:
# Prueba: policy evaluation = pi_DQN, policy behavior = estimated_pi_DQN
pruebas_ris(trajectories_memory,
            device_ris,
            pi_DQN,
            PATHS_NUMBERS,
            estimated_pi_DQN,
            action_type,
            HORIZON)

In [ ]:
# Prueba: policy evaluation = pi_NFQ, policy behavior = estimated_pi_DQN
pruebas_ris(trajectories_memory,
            device_ris,
            pi_NFQ,
            PATHS_NUMBERS,
            estimated_pi_DQN,
            action_type,
            HORIZON)